# Getting a machine ready to follow along

<hr/>

To follow along with this tutorial, you will need to be sure you have a working installation of [Stan](https://mc-stan.org/) and other tools we will use to process the results from our sampling in Stan. Alternatively, you can follow along in Google Colab. This notebook describes how to set up each.

## Configuring your own machine

In addition to staples like Numpy, SciPy, Pandas, Xarray, etc., we will be using the following packages

- `bokeh`: My preferring plotting package.
- `bebi103`: A set of utilities I use in my class on statistical inference.
- `iqplot`: A plotting package to plot data sets with one quantitative variable in Bokeh.
- `colorcet`: A package with many convenient color palettes.
- `arviz`: A package for processing MCMC samples.
- `cmdstanpy`: My preferred interface to Stan.

I have found setting up a conda environment to be the easiest way to ensure a group of people I am working with have the necessary installations. Therefore, if you have not already, I advise installing either [Miniconda](https://docs.conda.io/en/latest/miniconda.html) or [Anaconda](https://www.anaconda.com/download). 

We will use the conda environment I built for my class. You can download the YML file specifying the environment here: [bebi103.yml](https://raw.githubusercontent.com/justinbois/learnbayes-livecode/main/bebi103.yml). Upon downloading YML file, you can create the environment by entering the following on the command line (making sure you are in the directory where the `bebi103.yml` file is).

    conda env create -f bebi103.yml

Then, to activate the environment, you will need to do

    conda activate bebi103

Finally, you will need to make sure Stan is installed. You can do this by running

    python -c "import cmdstanpy; cmdstanpy.install_cmdstan()"

on the command line. Note that you will need to have a C++ toolchain installed on your machine for this to work.

## Use of Google Colab

The notebooks work in Google Colab. You will just need to be sure to execute the first cell of the notebooks, which installs the necessary software for your Colab instance, and looks like this:

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade iqplot colorcet datashader bebi103 arviz cmdstanpy"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    import cmdstanpy; cmdstanpy.install_cmdstan()

The installations take a while, so that cell may take several minutes to execute. You can run this notebook in Colab at [this link](https://colab.research.google.com/github/justinbois/learnbayes-livecode/blob/main/setup.ipynb). 

## Checking your installations

You can check your installation (or Colab) by running the cell below. You should see a Stan model get compiled, generation of samples, and finally a corner plot.

In [2]:
import numpy as np

import cmdstanpy
import arviz as az

import bebi103

import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

schools_data = {
    "J": 8,
    "y": [28, 8, -3, 7, -1, 1, 18, 12],
    "sigma": [15, 10, 16, 11, 9, 11, 10, 18],
}

schools_code = """
data {
  int<lower=0> J; // number of schools
  vector[J] y; // estimated treatment effects
  vector<lower=0>[J] sigma; // s.e. of effect estimates
}

parameters {
  real mu;
  real<lower=0> tau;
  vector[J] eta;
}

transformed parameters {
  vector[J] theta = mu + tau * eta;
}

model {
  eta ~ normal(0, 1);
  y ~ normal(theta, sigma);
}
"""

with open("schools_code.stan", "w") as f:
    f.write(schools_code)

sm = cmdstanpy.CmdStanModel(stan_file="schools_code.stan")
samples = sm.sample(data=schools_data, output_dir="./", show_progress=False)
samples = az.from_cmdstanpy(samples)

# Make a corner plot of samples
p = bebi103.viz.corner(samples, parameters=['mu', 'tau'])

bokeh.io.show(p)

Loading BokehJS ...

01:10:16 - cmdstanpy - INFO - compiling stan file /Users/bois/Dropbox/git/learnbayes-livecode/schools_code.stan to exe file /Users/bois/Dropbox/git/learnbayes-livecode/schools_code
01:10:29 - cmdstanpy - INFO - compiled model executable: /Users/bois/Dropbox/git/learnbayes-livecode/schools_code
01:10:29 - cmdstanpy - INFO - CmdStan start processing
01:10:29 - cmdstanpy - INFO - Chain [1] start processing
01:10:29 - cmdstanpy - INFO - Chain [2] start processing
01:10:29 - cmdstanpy - INFO - Chain [3] start processing
01:10:29 - cmdstanpy - INFO - Chain [4] start processing
01:10:29 - cmdstanpy - INFO - Chain [1] done processing
01:10:29 - cmdstanpy - INFO - Chain [2] done processing
01:10:29 - cmdstanpy - INFO - Chain [3] done processing
01:10:29 - cmdstanpy - INFO - Chain [4] done processing


## Computing environment

In [3]:
%load_ext watermark
%watermark -v -p numpy,bokeh,cmdstanpy,arviz,bebi103,jupyterlab
print("CmdStan : {0:d}.{1:d}".format(*cmdstanpy.cmdstan_version()))

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.12.0

numpy     : 1.24.3
bokeh     : 3.2.1
cmdstanpy : 1.1.0
arviz     : 0.16.1
bebi103   : 0.1.14
jupyterlab: 4.0.3

CmdStan : 2.32
